In [2]:
import pandas as pd
import camelot
from pathlib import Path


In [ ]:
p = Path(r"./data")

In [18]:
df = pd.DataFrame()
id = 0
for x in p.glob("*/Вакуумная система*"):
    #if x.parents[0].parts[-1].find("Шаг") ==-1:
    tables = camelot.read_pdf(x, pages="all")
    for i in tables:
        i.df.drop(0, inplace=True)
        i.df[8]=id
        df = pd.concat([df, i.df], axis=0)
    id+=1
    print(f"обработан: {x}")
data_vs  = df.dropna(axis=0).reset_index().drop("index", axis=1)
df = pd.DataFrame()
for x in p.glob("*/НГ0*"):
    #if x.parents[0].parts[-1].find("Шаг") ==-1:
    tables = camelot.read_pdf(x, pages="all")
    for i in tables:
        i.df.drop(0, inplace=True)
        df = pd.concat([df, i.df], axis=0)
    print(f"обработан: {x}")
data_ng0  = df.dropna(axis=0).reset_index().drop("index", axis=1)
df = pd.DataFrame()
for x in p.glob("*/НГ1*"):
    #if x.parents[0].parts[-1].find("Шаг") ==-1:
    tables = camelot.read_pdf(x, pages="all")
    for i in tables:
        i.df.drop(0, inplace=True)
        df = pd.concat([df, i.df], axis=0)
    print(f"обработан: {x}")
data_ng1  = df.dropna(axis=0).reset_index().drop("index", axis=1)

обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 17 12-35-04) (ID Служебная программа) (Атмосфера)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 17 13-53-06) (ID Служебная программа) (Атмосфера)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 17 15-31-27) (ID Служебная программа) (Вакуум)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 17 15-32-11) (ID Служебная программа) (Вакуум)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 18 09-41-07) (ID Служебная программа) (Атмосфера)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 18 10-31-21) (ID Служебная программа) (Вакуум)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 18 12-26-32) (ID Служебная программа) (Атмосфера)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 18 13-12-10) (ID Служебная программа) (Вакуум)\Вакуумная система.pdf
обработан: C:\Users\kaiko\Desktop\ВС\(2025 03 18 13-42-35) (ID Служе

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 38124944 bytes in function 'cv::OutOfMemoryError'


In [ ]:
data_ng1 = data_ng1[ ~data_ng1[0].str.contains("мальные")]
data_vs.to_csv("datavs.csv")
data_ng0.to_csv("datang0.csv")
data_ng1.to_csv("datang1.csv")

In [ ]:
data_ng0[0] = pd.to_datetime(data_ng0[0].str.replace("\n", "").str.replace(" марта ", "-03-").str.replace(" апреля ", "-04-").str.replace(" мая ", "-05-").str.replace(" апреля ", "-04-").str.replace(" г.", " "), dayfirst=True)
data_vs[0] = pd.to_datetime(data_vs[0], dayfirst=True)
data_ng1[0] = pd.to_datetime(data_ng1[0].str.replace("\n", "").str.replace(" июня ", "-06-").str.replace(" июля ", "-07-").str.replace(" августа ", "-08-").str.replace(" сентября ", "-09-").str.replace(" октября ", "-10-").str.replace(" ноября ", "-11-").str.replace(" г.", " "), dayfirst=True)


In [ ]:
data = pd.concat([data_vs.merge(data_ng0[[0, 1]], on=0, how="inner"), data_vs.merge(data_ng1[[0, 1]], left_on="0", right_on=0, how="inner")]).reset_index().drop("index", axis=1)
data.columns=["Date","Upper", "Lower", "Pressure", "Lg" , "Temp_NR1", "RVL", "NR1","id",  "Temp"]
for column in data.columns:
    if column != "Date" and data[column].dtype == "O":
        data[column] = data[column].str.replace(",", ".").astype("float")

In [127]:
data

,Date,Upper,Lower,Pressure,Lg,Temp_NR1,RVL,NR1,id,Temp
0,2025-03-17 12:35:05,0.005,0.0,73.28245,1.863,283.920,62805.836,6.637,0,0.0
1,2025-03-17 12:35:06,0.005,0.0,10702.86383,4.000,283.780,63095.734,6.637,0,0.0
2,2025-03-17 12:35:07,0.005,0.0,100000.00000,5.000,283.780,63241.185,6.637,0,0.0
3,2025-03-17 12:35:08,0.005,0.0,100000.00000,5.000,283.605,62805.836,6.637,0,0.0
4,2025-03-17 13:53:07,0.005,0.0,100000.00000,5.000,122.360,64565.423,0.714,1,0.0
...,...,...,...,...,...,...,...,...,...,...
232980,2025-11-14 15:28:58,0.003,0.0,0.00296,-2.538,10.045,82413.812,0.711,432,26.9
232981,2025-11-14 15:29:00,0.003,0.0,0.00296,-2.538,10.045,82413.812,0.711,432,26.9
232982,2025-11-14 15:29:02,0.003,0.0,0.00296,-2.538,10.045,82413.812,0.711,432,26.9
232983,2025-11-14 15:29:04,0.003,0.0,0.00296,-2.538,9.975,82035.154,0.711,432,26.9


In [128]:
data.to_csv("data.csv")